In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
import os

In [2]:
df = pd.read_csv('raw data/stock_prices.csv')

In [3]:
df.drop(labels=['AdjustmentFactor','ExpectedDividend', 'SupervisionFlag'], axis=1, inplace=True)

In [4]:
df.isnull().sum()

RowId                0
Date                 0
SecuritiesCode       0
Open              7608
High              7608
Low               7608
Close             7608
Volume               0
Target             238
dtype: int64

In [5]:
df2 = df.sort_values(['SecuritiesCode', 'Date'])

In [6]:
start = 0
end = 0
secu_code = df2.iloc[start]['SecuritiesCode']
for i in range(2332531):
    if df2.iloc[i]['SecuritiesCode']!=secu_code:
        end = i
        temp_df = pd.DataFrame(df2.iloc[start:end])
        temp_df.to_csv('processed data/'+str(secu_code)+'.csv', index = False)
        #print('%d length: %d'%(secu_code, start-end))
        start = i
        secu_code = df2.iloc[start]['SecuritiesCode']

temp_df = pd.DataFrame(df2.iloc[start:])
temp_df.to_csv('processed data/'+str(secu_code)+'.csv', index = False)

In [7]:
def feature_processing(file_name):
    df = pd.read_csv('processed data/' + file_name)
    df['pClose'] = df['Close'].shift(1)
    df['Return_Close'] = (df['Close'] - df['pClose'])/df['pClose'] 
    df['Return_Open'] = (df['Open'] - df['pClose'])/df['pClose']
    df['Return_High'] = (df['High'] - df['pClose'])/df['pClose']
    df['Return_Low'] = (df['Low'] - df['pClose'])/df['pClose']
    df.drop(labels = ['RowId', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close'], axis = 1, inplace = True)
    df.fillna(0, inplace = True)
    df.to_csv('train set/'+file_name, index=False)
    

In [8]:
sec_list = os.listdir('processed data')
for sec_code in sec_list:
    feature_processing(sec_code)

In [10]:
sec_list = os.listdir('train set')
print(len(sec_list))

2000


In [11]:
template_df = pd.read_csv('train set/9001.csv')

In [15]:
def fill_date(file_name, template_df):
    temp_df = template_df.copy(deep=True)
    df = pd.read_csv('train set/'+file_name)
    len_df = df.shape[0]
    p1 = 0
    p2 = 0
    while(p1<1202 and p2<len_df):
        if df.iloc[p2]['Date'] == temp_df.iloc[p1]['Date']:
            temp_df.iloc[p1, df.columns.get_loc('Return_Close')] = df.iloc[p2]['Return_Close']
            temp_df.iloc[p1, df.columns.get_loc('Return_Open')] = df.iloc[p2]['Return_Open']
            temp_df.iloc[p1, df.columns.get_loc('Return_High')] = df.iloc[p2]['Return_High']
            temp_df.iloc[p1, df.columns.get_loc('Return_Low')] = df.iloc[p2]['Return_Low']
            temp_df.iloc[p1, df.columns.get_loc('Target')] = df.iloc[p2]['Target']
            temp_df.iloc[p1, df.columns.get_loc('Volume')] = df.iloc[p2]['Volume']
            p1+=1
            p2+=1
        else:
            temp_df.iloc[p1, df.columns.get_loc('Return_Close')] = 0
            temp_df.iloc[p1, df.columns.get_loc('Return_Open')] = 0
            temp_df.iloc[p1, df.columns.get_loc('Return_High')] = 0
            temp_df.iloc[p1, df.columns.get_loc('Return_Low')] = 0
            temp_df.iloc[p1, df.columns.get_loc('Target')] = 0
            temp_df.iloc[p1, df.columns.get_loc('Volume')] = 0
            p1+=1
    while(p1<1202):
        temp_df.iloc[p1, df.columns.get_loc('Return_Close')] = 0
        temp_df.iloc[p1, df.columns.get_loc('Return_Open')] = 0
        temp_df.iloc[p1, df.columns.get_loc('Return_High')] = 0
        temp_df.iloc[p1, df.columns.get_loc('Return_Low')] = 0
        temp_df.iloc[p1, df.columns.get_loc('Target')] = 0
        temp_df.iloc[p1, df.columns.get_loc('Volume')] = 0
        p1+=1
    temp_df.to_csv('train set/'+file_name, index = False)

In [18]:
sec_list = os.listdir('train set')
if '.ipynb_checkpoints' in sec_list:
    sec_list.remove('.ipynb_checkpoints')
short_list = []
for sec_code in sec_list:
    df = pd.read_csv('train set/'+sec_code)
    if df.shape[0] < 1202:
        short_list.append(sec_code)
print(len(short_list))

135


In [19]:
for sec_code in short_list:
    fill_date(sec_code, template_df)

In [15]:
sec_list = os.listdir('train set')
short_list = []
for sec_code in sec_list:
    df = pd.read_csv('train set/'+sec_code)
    if df.shape[0] < 1202:
        short_list.append(sec_code)
print(len(short_list))

0
